In [1]:
import kagglehub
import pandas as pd
import numpy as np
import nltk
import sklearn
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import torch
print(torch.__version__)
from transformers import pipeline

# # Download latest version
# path = kagglehub.dataset_download("eshummalik/socialbuzz-sentiment-analytics")

# print("Path to dataset files:", path)

2.10.0+cpu


In [2]:
path = r"C:\Users\Kelby Mace\.cache\kagglehub\datasets\eshummalik\socialbuzz-sentiment-analytics\versions\4"

df = pd.read_csv(os.path.join(path, "sentimentdataset.csv"))

# Fix data tytpes
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Likes'] = df['Likes'].astype(int)
df['Retweets'] = df['Retweets'].astype(int)

df.head()

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19


In [ ]:
# Clean text

def clean_text(text):
    """Lowercase, strip whitespace, remove punctuation, and replace numbers with <NUM>."""
    text = text.lower()
    text = text.strip()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'[0-9]', '<NUM>', text)
    return text


# Apply text cleaning function
df['Text'] = df['Text'].apply(clean_text)
df['Sentiment'] = df['Sentiment'].apply(lambda x: x.strip().lower())

#Add text length feature
df['Text Length'] = df['Text'].apply(len)

# Drop extra index columns
if 'Unnamed: 0.1' in df.columns:
    df = df.drop(columns=['Unnamed: 0.1'])
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Text Length
0,enjoying a beautiful day at the park,positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12,36
1,traffic was terrible this morning,negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8,33
2,just finished an amazing workout,positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15,33
3,excited about the upcoming weekend getaway,positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18,42
4,trying out a new recipe for dinner tonight,neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19,42


In [6]:
df['Sentiment'].value_counts()

Sentiment
positive          45
joy               44
excitement        37
contentment       19
neutral           18
                  ..
lostlove           1
emotionalstorm     1
suffering          1
bittersweet        1
intrigue           1
Name: count, Length: 191, dtype: int64

In [ ]:
# Comment out after initial run - save off csv with cleaned text and new feature
# candidate = ["positive", "neutral", "negative"]
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# def map_label(label: str):
#     out = classifier(label, candidate_labels=candidate, multi_label=False)
#     return out["labels"][0], out["scores"][0]  # predicted class + confidence

# # Example
# map_label("furious")


In [5]:
# Make new label column from zero-shot classification with fewer categories

# df['Predicted Sentiment'] = df['Text'].apply(lambda x: map_label(x)[0])
# df['Prediction Confidence'] = df['Text'].apply(lambda x: map_label(x)[1])

# df.head()

In [16]:
df['Predicted Sentiment'].value_counts(normalize=True)

Predicted Sentiment
positive    0.684426
negative    0.278689
neutral     0.036885
Name: proportion, dtype: float64

In [18]:
df.loc[df['Prediction Confidence'] == df['Prediction Confidence'].min()]

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Text Length,Predicted Sentiment,Prediction Confidence
349,contemplating lifes mysteries under the starry...,contemplation,2016-07-02 23:30:00,Stargazer,Twitter,#Contemplation #StarryNight,28,55,UK,2016,7,2,23,56,negative,0.348642


In [ ]:
# df['Sentiment Reduced'], df['Sentiment Reduced Confidence'] = zip(*df['Sentiment'].apply(lambda x: map_label(x)))

In [ ]:
# df.to_csv("sentiment_with_predictions.csv", index=False)

In [ ]:
# For sentiments already classified as positive, neutral, or negative, how often does the predicted sentiment match the original sentiment?

df_three = df.copy()

df_three = df_three.loc[df_three['Sentiment'].isin(["positive", "neutral", "negative"])]
df_three['Text Sentiment Match'] = df_three.apply(lambda row: row['Sentiment'] == row['Predicted Sentiment'], axis=1)

# This works 100% of the time -- good gut check
df_three['Sentiment Prediction Match'] = df_three.apply(lambda row: row['Sentiment'] == row['Sentiment Reduced'], axis=1)

df_three['Predictions Match'] = df_three.apply(lambda row: row['Text Sentiment Match'] == row['Sentiment Prediction Match'], axis=1)

df_three['Text Sentiment Match'].value_counts(normalize=True), df_three['Sentiment Prediction Match'].value_counts(normalize=True), df_three['Predictions Match'].value_counts(normalize=True)

(Text Sentiment Match
 True     0.671642
 False    0.328358
 Name: proportion, dtype: float64,
 Sentiment Prediction Match
 True    1.0
 Name: proportion, dtype: float64,
 Predictions Match
 True     0.671642
 False    0.328358
 Name: proportion, dtype: float64)

In [10]:
df = pd.read_csv("sentiment_with_predictions.csv")

df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Text Length,Predicted Sentiment,Prediction Confidence,Sentiment Reduced,Sentiment Reduced Confidence
0,enjoying a beautiful day at the park,positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12,36,positive,0.936028,positive,0.987529
1,traffic was terrible this morning,negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8,33,negative,0.976853,negative,0.988335
2,just finished an amazing workout,positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15,32,positive,0.923207,positive,0.987529
3,excited about the upcoming weekend getaway,positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18,42,positive,0.975727,positive,0.987529
4,trying out a new recipe for dinner tonight,neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19,42,positive,0.662256,neutral,0.855826


In [4]:
df['Country'].value_counts()

Country
USA               59
USA               55
UK                49
Canada            44
Australia         41
                  ..
Netherlands        1
USA                1
Germany            1
France             1
USA                1
Name: count, Length: 115, dtype: int64

In [14]:
df['Hashtags'] = df['Hashtags'].apply(lambda x: x.strip().lower())
df['Hashtags'] = df['Hashtags'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))
df['Platform'] = df['Platform'].apply(lambda x: x.strip().lower())
df['Num Hashtags'] = df['Hashtags'].apply(lambda x: len(x.split()) if pd.notnull(x) else 0)

df.head()

,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,Text Length,Predicted Sentiment,Prediction Confidence,Sentiment Reduced,Sentiment Reduced Confidence,Num Hashtags
0,enjoying a beautiful day at the park,positive,2023-01-15 12:30:00,User123,twitter,nature park,15,30,USA,2023,1,15,12,36,positive,0.936028,positive,0.987529,2
1,traffic was terrible this morning,negative,2023-01-15 08:45:00,CommuterX,twitter,traffic morning,5,10,Canada,2023,1,15,8,33,negative,0.976853,negative,0.988335,2
2,just finished an amazing workout,positive,2023-01-15 15:45:00,FitnessFan,instagram,fitness workout,20,40,USA,2023,1,15,15,32,positive,0.923207,positive,0.987529,2
3,excited about the upcoming weekend getaway,positive,2023-01-15 18:20:00,AdventureX,facebook,travel adventure,8,15,UK,2023,1,15,18,42,positive,0.975727,positive,0.987529,2
4,trying out a new recipe for dinner tonight,neutral,2023-01-15 19:55:00,ChefCook,instagram,cooking food,12,25,Australia,2023,1,15,19,42,positive,0.662256,neutral,0.855826,2


In [16]:
# ----------------------------
# 1) Define target + features
# ----------------------------
target_col = "Sentiment Reduced"
# text_col = "Text"

# Optional: remove columns you do NOT want as predictors (IDs, leakage fields, etc.)
drop_cols = [
    target_col,
    "Sentiment",  # uncomment if this would leak target information
    "Predicted Sentiment",
    "Prediction Confidence",
    "Sentiment Reduced Confidence",
    "Timestamp",
    "User"
]

df_model = df.copy()
df_model = df_model.dropna(subset=[target_col, text_col])

X = df_model.drop(columns=[c for c in drop_cols if c in df_model.columns], errors="ignore")
y = df_model[target_col]

# Auto-detect other feature types (excluding text)
text_features = ["Text", "Hashtags"]
numeric_features = [c for c in X.select_dtypes(include=[np.number]).columns if c not in text_features]
categorical_features = [c for c in X.select_dtypes(include=["object", "category", "bool"]).columns if c not in text_features]

print("Text features:", text_features)
print("Numeric features:", numeric_features)
print("Categorical features:", categorical_features)

Text features: ['Text', 'Hashtags']
Numeric features: ['Retweets', 'Likes', 'Year', 'Month', 'Day', 'Hour', 'Text Length', 'Num Hashtags']
Categorical features: ['Platform', 'Country']


In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

# ----------------------------
# 2) Preprocessing
# ----------------------------

# Experiment 1: No stemming/lemmatization, stop word removal

custom_stopwords = set(text.ENGLISH_STOP_WORDS) - {"not", "no", "never"}
custom_stopwords = sorted(list(custom_stopwords))  # convert set -> list

text_transformer = TfidfVectorizer(
    stop_words=custom_stopwords,   # custom English stopword list
    ngram_range=(1,2),
    min_df=2
)

hashtags_transformer = TfidfVectorizer(
    stop_words="english",
    ngram_range=(1,2),
    min_df=2
)

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False))  # works with sparse output
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_transformer, "Text"),
        ("hashtags", hashtags_transformer, "Hashtags"),
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="drop"
)

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# ----------------------------
# 3) Model pipeline
# ----------------------------
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(max_iter=2000))
])

# ----------------------------
# 4) Train/test split + fit
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.93      0.61      0.74        44
     neutral       0.00      0.00      0.00         6
    positive       0.82      1.00      0.90        97

    accuracy                           0.84       147
   macro avg       0.58      0.54      0.55       147
weighted avg       0.82      0.84      0.82       147

Confusion matrix:
 [[27  0 17]
 [ 2  0  4]
 [ 0  0 97]]


c:\Users\Kelby Mace\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Kelby Mace\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Kelby M

In [ ]:
# Debug pass

from sklearn.model_selection import StratifiedKFold, GridSearchCV

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=2,
    error_score="raise"
)

grid.fit(X_train, y_train)



Fitting 5 folds for each of 486 candidates, totalling 2430 fits


c:\Users\Kelby Mace\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


,estimator,Pipeline(step...ver='saga'))])
,param_grid,"{'model__C': [0.5, 1.0, ...], 'preprocessor__hashtags__max_features': [1000, 3000, ...], 'preprocessor__hashtags__min_df': [1, 2, ...], 'preprocessor__text__max_features': [10000, 20000, ...], ...}"
,scoring,'f1_macro'
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,'raise'
,return_train_score,False
,transformers,"[('text', ...), ('hashtags', ...), ...]"


In [35]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Rebuild pipeline so model + preprocessing are in one object for CV tuning
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(
        max_iter=10000,
        class_weight=None,
        solver="saga",
        C=1.0
    ))
])

# 2) Stratified CV + optimize for macro F1
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
macro_f1 = make_scorer(f1_score, average="macro")

# 3) Reduce TF-IDF noise (tune min_df / max_features)
param_grid = {
    "preprocessor__text__min_df": [2, 3],
    "preprocessor__text__max_features": [10000, 20000],
    "preprocessor__hashtags__min_df": [1, 2],
    "preprocessor__hashtags__max_features": [1000, 3000],
    "model__C": [0.5, 1.0],
}



grid = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=2,
    error_score="raise"
)

grid.fit(X_train, y_train)

print("Best CV macro F1:", grid.best_score_)
print("Best params:", grid.best_params_)

best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0))


Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best CV macro F1: 0.4142651536369873
Best params: {'model__C': 0.5, 'preprocessor__hashtags__max_features': 1000, 'preprocessor__hashtags__min_df': 2, 'preprocessor__text__max_features': 10000, 'preprocessor__text__min_df': 3}
              precision    recall  f1-score   support

    negative       0.55      0.25      0.34        44
     neutral       0.00      0.00      0.00         6
    positive       0.72      0.94      0.81        97

    accuracy                           0.69       147
   macro avg       0.42      0.40      0.39       147
weighted avg       0.64      0.69      0.64       147

